# YOLOv8 Pipeline on MinneApple Dataset

This notebook demonstrates how to:
1. Install YOLOv8 and PyTorch
2. Mount Google Drive and copy YOLOv8 project
3. Data Acquisition and Preprocessing
4. Convert COCO ground-truth annotations to YOLO format
5. Train and validate the YOLOv8 model
6. Perform inference on new images

## 1. **Install Dependencies**

In [ ]:
!pip install ultralytics
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


## 2. **Mount Google Drive**


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. **Extraction of Minne Apple Dataset**
[MinneApple Dataset Link](https://rsn.umn.edu/projects/orchard-monitoring/minneapple)

Classify the folders in the Yolo format for training and validation, using images and labels respectively

In [ ]:
!unzip /content/drive/MyDrive/archive.zip -d /content/drive/MyDrive/archive


## 4. **Code for YOLO Labels Generation from masks**


In [ ]:
import os
import cv2
from glob import glob

masks_path = "/content/drive/MyDrive/MinneApple/MinneApple/detection/train/masks"
image_path = "/content/drive/MyDrive/MinneApple/MinneApple/detection/train/images"
labels_path = "/content/drive/MyDrive/MinneApple/MinneApple/detection/train/labels"

mask_files = glob(os.path.join(masks_path, "*.png"))
print(f"Number of mask files found: {len(mask_files)}")
print("Mask files:", mask_files[:10])  # Print the first 10 mask file names for verifying
os.makedirs(labels_path, exist_ok=True)

#To find all the .png files in masks folder

In [ ]:

for each_masks_path in mask_files:
    mask = cv2.imread(each_masks_path, cv2.IMREAD_GRAYSCALE) #0-255 - cv2.imread() defaultly reads image in BGR
    if mask is None:
      print(f"Failed path: {each_masks_path}")
      continue
    #RETR_EXTERNAL to ignore nested contours
    #CHAIN_APPROX_SIMPLE to store essential contour points
    boundaries, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print(f"{each_masks_path} - Number of contours found: {len(boundaries)}")

    annotations_yolo = []
    for boundary in boundaries:
      x, y, w, h = cv2.boundingRect(boundary)   #x, y Rectangle top left   #w- width, h - height

      height, width = mask.shape  #Dimensions
      horizonal_center = (x + w / 2)/width   #Centers of rectangle - fraction of img size
      vertical_center =  (y + h / 2)/height
      width_fraction = w/width #Normalization of bounding box # \cite "You Only Look Once: Unified, Real-Time Object Detection."
      height_fraction = h/height

      annotations_yolo.append(f"0 {horizonal_center} {vertical_center} {width_fraction} {height_fraction}") # 0 = class ID for Apple



    labels_file_name = os.path.basename(each_masks_path).replace(".png",".txt")  # mask to .txt
    labels_file_path = os.path.join(labels_path, labels_file_name)

    with open(labels_file_path,"w") as file:   #Saving annotations to labels_file_path
      file.write("\n".join(annotations_yolo)) #New line each # for annotation in annotations_yolo:

print(f"Labels Saved: {labels_file_name}")
print(f"Mask label file name: {labels_file_path}")

Checking the count of images and labels in the dataset for trainging

In [ ]:
train_images_count = len([file for file in os.listdir(image_path) if file.endswith(".png")])
train_labels_count = len([file for file in os.listdir(labels_path) if file.endswith(".txt")])

# Print counts
print(f"Number of train images: {train_images_count}")
print(f"Number of train labels: {train_labels_count}")

## **5. Splitting images in train to 80% train, 20%val**

In [ ]:
import os
from glob import glob
import shutil
import random

train_image_path = "/content/drive/MyDrive/MinneApple/yolov8/final_data/train/images"
train_labels_path = "/content/drive/MyDrive/MinneApple/yolov8/final_data/train/labels"
val_images_path = "/content/drive/MyDrive/MinneApple/yolov8/final_data/val/images"
val_labels_path = "/content/drive/MyDrive/MinneApple/yolov8/final_data/val/labels"

os.makedirs(val_images_path, exist_ok=True)
os.makedirs(val_labels_path, exist_ok=True)

image_files = glob(os.path.join(train_image_path, "*.jpg"))
random.shuffle(image_files)

index_split_division = int(0.9 * len(image_files))
training_files = image_files[:index_split_division]
validation_files = image_files[index_split_division:]

for image_val in validation_files:
  shutil.move(image_val, os.path.join(val_images_path, os.path.basename(image_val))) #To move images

  label_folder = os.path.join(train_labels_path, os.path.basename(image_val).replace(".jpg",".txt"))
  if os.path.exists(label_folder):
    shutil.move(label_folder, os.path.join(val_labels_path, os.path.basename(label_folder)))

print(f"{len(validation_files)} moved to val")


# The Datsets used include:
1. **MinneApple Dataset**
2. **Kaggle Dataset**
3. **Custom annotated dataset using Roboflow**

## **Code to Count files in each folder**



In this training the total count was:

1.   Number of train images: 2198
2.   Number of train labels: 2184
3.   Number of validation images: 318
4.   Number of validation labels: 317

In [ ]:
train_images_count = len([file for file in os.listdir(train_image_path) if file.endswith((".jpg",".png"))])
train_labels_count = len([file for file in os.listdir(train_labels_path) if file.endswith(".txt")])
val_images_count = len([file for file in os.listdir(val_images_path) if file.endswith((".jpg",".png"))])
val_labels_count = len([file for file in os.listdir(val_labels_path) if file.endswith(".txt")])

# Print counts
print(f"train images: {train_images_count}")
print(f"train labels: {train_labels_count}")
print(f"validation images: {val_images_count}")
print(f"validation labels: {val_labels_count}")

## Code to copy img and txt from Minne to final data
 Mix of MINNE+KAGGLE because both contain large number of images

In [ ]:
import os
import shutil

# Define source and destination folders
folders = [
    {
        "images": "/content/drive/MyDrive/MinneApple/yolov8/dataset/images/train",
        "labels": "/content/drive/MyDrive/MinneApple/yolov8/dataset/labels/train"
     },
    {
        "images": "/content/drive/MyDrive/MinneApple/yolov8/dataset/images/val",
        "labels": "/content/drive/MyDrive/MinneApple/yolov8/dataset/labels/val"
     }
]

destination_folders = [
    {
        "images": "/content/drive/MyDrive/MinneApple/yolov8/final_data/train/images",
        "labels": "/content/drive/MyDrive/MinneApple/yolov8/final_data/train/labels"
        },
    {
        "images": "/content/drive/MyDrive/MinneApple/yolov8/final_data/val/images",
        "labels": "/content/drive/MyDrive/MinneApple/yolov8/final_data/val/labels"
    }
]
# Ensure destination folders exist
for dest in destination_folders:
    os.makedirs(dest["images"], exist_ok=True)
    os.makedirs(dest["labels"], exist_ok=True)

def copy_files(src_folder, dest_folder, file_extension):
    """
    Copies files with a specific extension from the source to the destination folder.
    """
    for root, _, files in os.walk(src_folder):
        for file in files:
            if file.endswith(file_extension):
                src_file = os.path.join(root, file)
                dest_file = os.path.join(dest_folder, file)
                 # Print source and destination file paths for confirmation
                print(f"Copying: {src_file} -> {dest_file}")
                shutil.copy2(src_file, dest_file)
# Copy images and labels
for i in range(len(folders)):
    src = folders[i]
    dest = destination_folders[i]
    # copy_files(src["images"], dest["images"], ".jpg")
    copy_files(src["images"], dest["images"], ".png")
    copy_files(src["labels"], dest["labels"], ".txt")

print("Files copied successfully!")


## **Code to check and verify missing label image pair**

In [ ]:
import os

# Paths
val_images_dir = "/content/drive/MyDrive/MinneApple/MinneApple/detection/val/images"
val_labels_dir = "/content/drive/MyDrive/MinneApple/MinneApple/detection/val/labels"

# Get list of image and label files
val_images = {os.path.splitext(f)[0] for f in os.listdir(val_images_dir) if f.endswith(".png")}
val_labels = {os.path.splitext(f)[0] for f in os.listdir(val_labels_dir) if f.endswith(".txt")}

# Find unmatched files
missing_images = val_labels - val_images  # Labels without images
missing_labels = val_images - val_labels  # Images without labels

print(f"Labels without images: {missing_images}")
print(f"Images without labels: {missing_labels}")

# **Training**

## 3. **Check CUDA & GPU**


In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

!nvidia-smi


## 4. **Train YOLOv8 Model**

Below, train using the `ultralytics` package.
Adjust `data`, `epochs`, `batch`, `imgsz`, and `device` as needed.


In [ ]:
from ultralytics import YOLO

# Load a pretrained YOLOv8 model (e.g., yolov8n, yolov8s, yolov8m, yolov8l, yolov8x)
model = YOLO('yolov8s.pt')

# Train the model
model.train(
    data="/content/drive/MyDrive/MinneApple/yolov8/final_data/final.yaml",  # Path to dataset config
    epochs=50,
    batch=16,
    imgsz=640,
    project="/content/drive/MyDrive/MinneApple/yolov8/final_data",  # Where results are saved
    name="combined_model",
    device=0  # 0 for GPU, or 'cpu' for CPU
)

## 5. **Check Missing Labels or Images**

This script checks if there are any images without corresponding annotated label files or vice versa.


In [ ]:
import os

val_images_path = '/content/drive/MyDrive/MinneApple/yolov8/data/images/val'
val_labels_path = '/content/drive/MyDrive/MinneApple/yolov8/data/labels/val'

image_files = [os.path.splitext(f)[0] for f in os.listdir(val_images_path) if f.endswith(('.png', '.jpg'))]
label_files = [os.path.splitext(f)[0] for f in os.listdir(val_labels_path) if f.endswith('.txt')]

missing_labels = [img for img in image_files if img not in label_files]
missing_images = [label for label in label_files if label not in image_files]

print(f"Images without labels: {missing_labels}")
print(f"Labels without images: {missing_images}")


## 6. **Alternative YOLOv8 Training (CLI Command)**

Training via the `yolo` CLI. Example 1-epoch training just as a demo.


In [ ]:
!yolo task=detect mode=train model=yolov8n.pt \
    data=/content/drive/MyDrive/MinneApple/yolov8/final_data/final.yaml \
    epochs=1 batch=16 imgsz=640 \
    project=/content/drive/MyDrive \
    name=result_expt


## 9. **Inference on Trained YOLOv8s Model**

Below inference is run on YOLOv8 model trained using a combination of 3 datasets of 3000 images






In [ ]:
 @misc{hani2019minneapple,
    title={MinneApple: A Benchmark Dataset for Apple Detection and Segmentation},
    author={Nicolai Häni and Pravakar Roy and Volkan Isler}
    year={2019},
    eprint={1909.06441},
    archivePrefix={arXiv},
    primaryClass={cs.CV}
}

In [ ]:
from ultralytics import YOLO

# Load the trained model (combined)
model_combined = YOLO("/content/drive/MyDrive/MinneApple/yolov8/final_data/combined_model/weights/best.pt")

# Run inference on a list of images
results_combined = model_combined(["/content/drive/MyDrive/yolov8/dataset1_back_181.png"])

# Display & save results
for result in results_combined:
    result.show()
    result.save("result_combined.jpg")

## 10. **Inference with Minne-only Model**

Here, a model trained only on MinneApple dataset is loaded for comparison


In [ ]:
model_minne = YOLO("/content/drive/MyDrive/yolov8/runs/train/minneapple2/best (2).pt")

results_minne = model_minne(["/content/drive/MyDrive/yolov8/dataset1_back_181.png"])
for result in results_minne:
    result.show()
    result.save("result_minne.jpg")


## 12. **Convert COCO Ground Truth to YOLO Format**

This section converts `ground_truth.json` (COCO format) into YOLO label files.


In [ ]:
import json
import os

coco_json_path = "/content/drive/MyDrive/yolov8/ground_truth.json"
output_dir = "output_labels"
os.makedirs(output_dir, exist_ok=True)

with open(coco_json_path, 'r') as file:
    coco_data = json.load(file)

image_id_to_filename = {image['id']: image['filename'] for image in coco_data['images']}
image_id_to_size = {image['id']: (image['width'], image['height']) for image in coco_data['images']}

for annotation in coco_data['annotations']:
    image_id = annotation['image_id']
    bbox = annotation['bbox']
    category_id = annotation['category_id']
    width, height = image_id_to_size[image_id]

    x_center = (bbox[0] + bbox[2] / 2) / width
    y_center = (bbox[1] + bbox[3] / 2) / height
    bbox_width = bbox[2] / width
    bbox_height = bbox[3] / height

    # YOLO label line
    yolo_line = f"{category_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n"

    # Change extension to .txt
    filename = image_id_to_filename[image_id].replace('.png', '.txt')
    label_path = os.path.join(output_dir, filename)

    # Append (in case multiple annotations exist for one image)
    with open(label_path, 'a') as label_file:
        label_file.write(yolo_line)

print(f"Labels have been saved to {output_dir}")


# **Results**
mAP@50, PR curve, F1 curve, Normalization matrix generated during this processs can be used to evaluate the perfrmance of the model. Further adding more annotated images and increasing the number of training epochs will help imrpove the model metrics

## **Conclusion**

**Installing Dependencies** – Set up YOLOv8, PyTorch, and Google Colab environment.

**Mounting Google Drive** – Load dataset and project files from Drive.

**Data Acquisition & Preprocessing** – Extract MinneApple dataset and generate YOLO labels from segmentation masks.

**Dataset Splitting & Organization**– Classified data into train and validation sets (80-20 split).

**Dataset Combination** – Merged MinneApple, Kaggle, and custom datasets for improved model robustness.

**Data Verification & Quality Check** – Ensured each image has a corresponding label file.

**Training YOLOv8 Model** – Used 50 epochs, batch size 16, and 640×640 images for detection.

**Validating Training Results** – Analyzed loss curves and mAP (mean Average Precision).

**Inference on Trained YOLOv8 Model** – Compared model performance on MinneApple-only vs. combined dataset.

**Ground Truth Annotations Conversion** – Converted COCO JSON annotations to YOLO format.

**Next Steps**:
1. Tweak hyperparameters (epochs, batch size, etc.) for better accuracy.
2. Use more advanced YOLOv8 variants (`yolov8m`, `yolov8l`) if GPU allows.
3. Share or push this notebook to GitHub to showcase work.
